In [1]:
import os
import sys
from pathlib import Path
import pandas as pd

# Load Data

In [2]:
gnss_trace_1_pd = pd.read_csv(r'C:\Users\nguye\Documents\GitHub\emtl30kr_gnss_logger\data\20250725_katech\p10_em_20km.csv')
gnss_trace_2_pd = pd.read_csv(r'C:\Users\nguye\Documents\GitHub\emtl30kr_gnss_logger\data\20250725_katech\p10_f9k_20km.csv')

# Pre-processing Data

In [3]:
# filter out unwanted fix types
filterd_fix_types = [
    # 'no-fix',
    # 'sps',
    # 'dgps',
    # 'pps',
    'fixed-rtk',
    # 'float-rtk',
    'dead-reckoning',
    # 'extrapolated',
]

filtered_trace_1_pd = gnss_trace_1_pd[gnss_trace_1_pd['FixType'].isin(filterd_fix_types)]
filtered_trace_2_pd = gnss_trace_2_pd[gnss_trace_2_pd['FixType'].isin(filterd_fix_types)]

# filter out unwanted columns
filtered_columns = ['GNSSTime', 'Latitude', 'Longitude', 'FixType']

filtered_trace_1_pd = filtered_trace_1_pd[filtered_columns]
filtered_trace_2_pd = filtered_trace_2_pd[filtered_columns]


In [4]:
# Trim data to have the closest start and end times
start_time = max(filtered_trace_1_pd['GNSSTime'].min(), filtered_trace_2_pd['GNSSTime'].min())
end_time = min(filtered_trace_1_pd['GNSSTime'].max(), filtered_trace_2_pd['GNSSTime'].max())

filtered_trace_1_pd = filtered_trace_1_pd[(filtered_trace_1_pd['GNSSTime'] >= start_time) & (filtered_trace_1_pd['GNSSTime'] <= end_time)]
filtered_trace_2_pd = filtered_trace_2_pd[(filtered_trace_2_pd['GNSSTime'] >= start_time) & (filtered_trace_2_pd['GNSSTime'] <= end_time)]

# Visualize Traces

In [5]:
import folium
import numpy as np

In [6]:
filtered_trace_1_np = filtered_trace_1_pd[['Latitude', 'Longitude']].to_numpy()
filetered_trace_2_np = filtered_trace_2_pd[['Latitude', 'Longitude']].to_numpy()

# Calculate map center (mean of first trace, or combine for both)
center = [filtered_trace_1_np[:, 0].mean(), filtered_trace_1_np[:, 1].mean()]

# Create Folium map
m = folium.Map(location=center, zoom_start=17, tiles='OpenStreetMap', control_scale=True)
folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    overlay=False,
    control=True
).add_to(m)

# Trace 1: Blue
coordinates1 = list(map(tuple, filtered_trace_1_np))
folium.PolyLine(coordinates1, color='blue', weight=3, opacity=0.8, tooltip='Trace 1').add_to(m)
folium.Marker(coordinates1[0], popup='Start 1', icon=folium.Icon(color='green')).add_to(m)
folium.Marker(coordinates1[-1], popup='End 1', icon=folium.Icon(color='red')).add_to(m)

# Trace 2: Orange
coordinates2 = list(map(tuple, filetered_trace_2_np))
folium.PolyLine(coordinates2, color='orange', weight=3, opacity=0.8, tooltip='Trace 2').add_to(m)
folium.Marker(coordinates2[0], popup='Start 2', icon=folium.Icon(color='purple')).add_to(m)
folium.Marker(coordinates2[-1], popup='End 2', icon=folium.Icon(color='darkred')).add_to(m) 

m

# Compare using DWT

In [7]:
from dtw import dtw
from haversine import haversine

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [13]:
# Fix typo in variable name
filtered_trace_1_np = filtered_trace_1_pd[['Latitude', 'Longitude']].to_numpy()
filtered_trace_2_np = filtered_trace_2_pd[['Latitude', 'Longitude']].to_numpy()

# Downsample for speed
downsample_rate = 10
filtered_trace_1_np = filtered_trace_1_np[::downsample_rate]
filtered_trace_2_np = filtered_trace_2_np[::downsample_rate]

# Haversine distance (ensure you have the function)
def gnss_distance(p1, p2):
    # p1 and p2 are (lat, lon) numpy arrays
    return haversine((p1[0], p1[1]), (p2[0], p2[1]), unit='m')

# Run DTW
alignment = dtw(
    filtered_trace_1_np, 
    filtered_trace_2_np,
    dist_method=gnss_distance,
    keep_internals=True)

dtw_distance = alignment.distance
normalized_distance = alignment.normalizedDistance

print(f"DTW distance: {dtw_distance:.2f} m")
print(f"Normalized DTW distance: {normalized_distance:.2f} m")

# The optimal alignment path (indices)
indices = list(zip(alignment.index1, alignment.index2))
print("Optimal warping path (index pairs):", indices)

# Compute haversine distance at each alignment step
cost_vector = [
    gnss_distance(filtered_trace_1_np[i], filtered_trace_2_np[j])
    for i, j in indices
]
print("Cost along the optimal path:", cost_vector)


DTW distance: 67.51 m
Normalized DTW distance: 0.54 m
Optimal warping path (index pairs): [(np.int64(0), np.int64(0)), (np.int64(1), np.int64(0)), (np.int64(2), np.int64(0)), (np.int64(3), np.int64(0)), (np.int64(4), np.int64(0)), (np.int64(5), np.int64(1)), (np.int64(6), np.int64(2)), (np.int64(7), np.int64(3)), (np.int64(8), np.int64(4)), (np.int64(9), np.int64(5)), (np.int64(10), np.int64(6)), (np.int64(11), np.int64(7)), (np.int64(12), np.int64(8)), (np.int64(13), np.int64(9)), (np.int64(13), np.int64(10)), (np.int64(14), np.int64(11)), (np.int64(15), np.int64(12)), (np.int64(16), np.int64(13)), (np.int64(17), np.int64(14)), (np.int64(17), np.int64(15)), (np.int64(18), np.int64(16)), (np.int64(19), np.int64(17)), (np.int64(20), np.int64(18)), (np.int64(21), np.int64(19)), (np.int64(22), np.int64(20)), (np.int64(23), np.int64(21)), (np.int64(24), np.int64(22)), (np.int64(25), np.int64(22)), (np.int64(25), np.int64(23)), (np.int64(26), np.int64(23)), (np.int64(27), np.int64(24)), (np

# Compare Point-to-Point using Time-Sync with Interpolation

In [14]:
def sync_gnss(df_a, df_b, time_col='GNSSTime'):
    """
    Synchronizes two GNSS DataFrames within their common time window and sets
    the FixType of BOTH output DataFrames to 'synced'.

    Args:
        df_a (pd.DataFrame): The reference DataFrame.
        df_b (pd.DataFrame): The DataFrame to be realigned and interpolated.
        time_col (str): The name of the column containing the timestamps.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame] or tuple[None, None]:
            A tuple containing two synchronized DataFrames (synced_a, synced_b).
            Both will have their 'FixType' column set to 'synced'.
            Returns (None, None) if there is no overlapping time period.
    """
    # --- 1. Prepare DataFrames ---
    df_a_proc = df_a.copy()
    df_b_proc = df_b.copy()

    # Process each dataframe
    processed_dfs = []
    for i, df in enumerate([df_a_proc, df_b_proc]):
        # Skip rows where GNSSTime is None or NaN
        df = df[df[time_col].notna()]
        df = df[df[time_col] != 'None']  # Remove string 'None' values
        
        if df.empty:
            print(f"Warning: Dataset {i+1} has no valid {time_col} values")
            return None, None
            
        # Convert time strings to seconds since midnight for easier comparison
        # This preserves the original format while making it numeric for pandas operations
        def time_to_seconds(time_str):
            try:
                # Parse time string like "08:42:01.500000"
                time_parts = time_str.split(':')
                hours = int(time_parts[0])
                minutes = int(time_parts[1])
                seconds_parts = time_parts[2].split('.')
                seconds = int(seconds_parts[0])
                microseconds = int(seconds_parts[1]) if len(seconds_parts) > 1 else 0
                
                total_seconds = hours * 3600 + minutes * 60 + seconds + microseconds / 1000000
                return total_seconds
            except:
                return None
        
        # Convert time to numeric format
        df['time_numeric'] = df[time_col].apply(time_to_seconds)
        df = df[df['time_numeric'].notna()]  # Remove any that couldn't be converted
        
        if df.empty:
            print(f"Warning: Dataset {i+1} has no parseable time values")
            return None, None
            
        df.set_index('time_numeric', inplace=True)
        df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
        df.sort_index(inplace=True)
        
        processed_dfs.append(df)

    df_a_proc, df_b_proc = processed_dfs

    if df_a_proc.empty or df_b_proc.empty:
        print("Warning: One or both datasets are empty after cleaning.")
        return None, None

    # --- 2. Determine Common Time Window ---
    start_time_a = df_a_proc.index.min()
    end_time_a = df_a_proc.index.max()
    
    start_time_b = df_b_proc.index.min()
    end_time_b = df_b_proc.index.max()

    common_start_time = max(start_time_a, start_time_b)
    common_end_time = min(end_time_a, end_time_b)

    # Convert back to time format for display
    def seconds_to_time(seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = seconds % 60
        return f"{hours:02d}:{minutes:02d}:{secs:06.3f}"

    print(f"Dataset A time range: {seconds_to_time(start_time_a)} to {seconds_to_time(end_time_a)}")
    print(f"Dataset B time range: {seconds_to_time(start_time_b)} to {seconds_to_time(end_time_b)}")
    print(f"Common time window: {seconds_to_time(common_start_time)} to {seconds_to_time(common_end_time)}")

    if common_start_time >= common_end_time:
        print("Warning: No overlapping time window found.")
        return None, None

    # --- 3. Trim to Common Window using boolean indexing ---
    df_a_trimmed = df_a_proc[(df_a_proc.index >= common_start_time) & (df_a_proc.index <= common_end_time)]
    df_b_trimmed = df_b_proc[(df_b_proc.index >= common_start_time) & (df_b_proc.index <= common_end_time)]
    
    if df_a_trimmed.empty or df_b_trimmed.empty:
        print("Warning: No data in common time window after trimming.")
        return None, None
    
    synced_a = df_a_trimmed

    # --- 4. Interpolate and Create synced_b ---
    combined_index = synced_a.index.union(df_b_trimmed.index)
    df_b_reindexed = df_b_trimmed.reindex(combined_index)

    # Use linear interpolation
    numeric_cols = df_b_reindexed.select_dtypes(include=np.number).columns.tolist()
    # Remove the time_numeric column from interpolation since it's our index
    numeric_cols = [col for col in numeric_cols if col != 'time_numeric']
    df_b_reindexed[numeric_cols] = df_b_reindexed[numeric_cols].interpolate(method='linear')

    synced_b = df_b_reindexed.reindex(synced_a.index)

    # --- Set FixType for BOTH synchronized DataFrames ---
    for df in [synced_a, synced_b]:
        if 'FixType' in df.columns:
            df.loc[:, 'FixType'] = 'synced'

    print(f"Synchronized datasets: {len(synced_a)} and {len(synced_b)} records")
    return synced_a, synced_b

In [16]:
synced_trace_1, synced_trace_2 = sync_gnss(
    filtered_trace_1_pd, 
    filtered_trace_2_pd)

Dataset A time range: 10:42:26.200 to 10:43:01.110
Dataset B time range: 10:42:26.170 to 10:43:01.110
Common time window: 10:42:26.200 to 10:43:01.110
Synchronized datasets: 653 and 653 records
